<a href="https://colab.research.google.com/github/aksharat/Medical-Projects/blob/main/WarmUp/WarmUp5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip "/content/Segmentation01.zip"

Archive:  /content/Segmentation01.zip
replace Segmentation01/list_test.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import os, cv2
import numpy as np
import pandas as pd
import random, tqdm
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import albumentations as album

In [ ]:
DATA_DIR = '/content/Segmentation01'

traindata_df = pd.read_csv(os.path.join(DATA_DIR, 'list_train.txt'))
traindata_df

,.\train\org\1.png,.\train\label\1.png
0,.\train\org\2.png,.\train\label\2.png
1,.\train\org\3.png,.\train\label\3.png
2,.\train\org\4.png,.\train\label\4.png
3,.\train\org\5.png,.\train\label\5.png
4,.\train\org\6.png,.\train\label\6.png
5,.\train\org\7.png,.\train\label\7.png
6,.\train\org\8.png,.\train\label\8.png
7,.\train\org\9.png,.\train\label\9.png
8,.\train\org\10.png,.\train\label\10.png
9,.\train\org\11.png,.\train\label\11.png


In [ ]:
train_loader = DataLoader(traindata_df, batch_size=16, shuffle=True, num_workers=4)
# valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False, num_workers=4)

SEGMENTATION

In [ ]:
import os
import numpy as np
import cv2
from torch.utils.data import Dataset
from torchvision import transforms

class LungSegmentationDataset(Dataset):
    def __init__(self, img_dir, mask_dir, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(img_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.images[idx])
        mask_name = os.path.join(self.mask_dir, self.images[idx])  # Assuming mask names are the same as image names

        image = cv2.imread(img_name, cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(mask_name, cv2.IMREAD_GRAYSCALE)

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask

# Define the transformation to tensor
transform = transforms.Compose([
    transforms.ToTensor()
])

# Update with your actual directories
img_dir = '/content/Segmentation01/train/org'
mask_dir = '/content/Segmentation01/train/label'

# Create the dataset
dataset = LungSegmentationDataset(img_dir, mask_dir, transform=transform)


In [ ]:
# original unet which works well


import torch
import torch.nn as nn

class UNet(nn.Module):
    def __init__(self, num_classes):
        super(UNet, self).__init__()
        self.num_classes = num_classes
        self.contracting_11 = self.conv_block(in_channels=1, out_channels=64)
        self.contracting_12 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.contracting_21 = self.conv_block(in_channels=64, out_channels=128)
        self.contracting_22 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.contracting_31 = self.conv_block(in_channels=128, out_channels=256)
        self.contracting_32 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.contracting_41 = self.conv_block(in_channels=256, out_channels=512)
        self.contracting_42 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.middle = self.conv_block(in_channels=512, out_channels=1024)
        self.expansive_11 = nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_12 = self.conv_block(in_channels=1024, out_channels=512)
        self.expansive_21 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_22 = self.conv_block(in_channels=512, out_channels=256)
        self.expansive_31 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_32 = self.conv_block(in_channels=256, out_channels=128)
        self.expansive_41 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_42 = self.conv_block(in_channels=128, out_channels=64)
        self.output = nn.Conv2d(in_channels=64, out_channels=num_classes, kernel_size=3, stride=1, padding=1)

    def conv_block(self, in_channels, out_channels):
        block = nn.Sequential(nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(num_features=out_channels),
                                    nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(num_features=out_channels))
        return block

    def forward(self, X):
        contracting_11_out = self.contracting_11(X) # [-1, 64, 256, 256]
        contracting_12_out = self.contracting_12(contracting_11_out) # [-1, 64, 128, 128]
        contracting_21_out = self.contracting_21(contracting_12_out) # [-1, 128, 128, 128]
        contracting_22_out = self.contracting_22(contracting_21_out) # [-1, 128, 64, 64]
        contracting_31_out = self.contracting_31(contracting_22_out) # [-1, 256, 64, 64]
        contracting_32_out = self.contracting_32(contracting_31_out) # [-1, 256, 32, 32]
        contracting_41_out = self.contracting_41(contracting_32_out) # [-1, 512, 32, 32]
        contracting_42_out = self.contracting_42(contracting_41_out) # [-1, 512, 16, 16]
        middle_out = self.middle(contracting_42_out) # [-1, 1024, 16, 16]
        expansive_11_out = self.expansive_11(middle_out) # [-1, 512, 32, 32]
        expansive_12_out = self.expansive_12(torch.cat((expansive_11_out, contracting_41_out), dim=1)) # [-1, 1024, 32, 32] -> [-1, 512, 32, 32]
        expansive_21_out = self.expansive_21(expansive_12_out) # [-1, 256, 64, 64]
        expansive_22_out = self.expansive_22(torch.cat((expansive_21_out, contracting_31_out), dim=1)) # [-1, 512, 64, 64] -> [-1, 256, 64, 64]
        expansive_31_out = self.expansive_31(expansive_22_out) # [-1, 128, 128, 128]
        expansive_32_out = self.expansive_32(torch.cat((expansive_31_out, contracting_21_out), dim=1)) # [-1, 256, 128, 128] -> [-1, 128, 128, 128]
        expansive_41_out = self.expansive_41(expansive_32_out) # [-1, 64, 256, 256]
        expansive_42_out = self.expansive_42(torch.cat((expansive_41_out, contracting_11_out), dim=1)) # [-1, 128, 256, 256] -> [-1, 64, 256, 256]
        output_out = self.output(expansive_42_out) # [-1, num_classes, 256, 256]
        return output_out
# Initialize the model
model = UNet(2)


In [ ]:
from torch.utils.data import DataLoader
import torch.optim as optim

# Set your device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()  # Adjust the loss based on your requirements
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define data loader
batch_size = 16  # Adjust based on your resources and dataset size
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Train the model
num_epochs = 10  # Adjust based on your requirements

for epoch in range(num_epochs):
    for i, (images, masks) in enumerate(train_loader):
        images = images.to(device)
        masks = masks.to(device)

        # Forward pass
        outputs = model(images)
        print(outputs.shape,masks.shape)
        masks = masks.squeeze(1)
        masks = masks.long()
        loss = criterion(outputs, masks)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')


torch.Size([16, 2, 256, 256]) torch.Size([16, 1, 256, 256])
torch.Size([16, 2, 256, 256]) torch.Size([16, 1, 256, 256])
torch.Size([16, 2, 256, 256]) torch.Size([16, 1, 256, 256])
torch.Size([2, 2, 256, 256]) torch.Size([2, 1, 256, 256])
torch.Size([16, 2, 256, 256]) torch.Size([16, 1, 256, 256])
torch.Size([16, 2, 256, 256]) torch.Size([16, 1, 256, 256])
torch.Size([16, 2, 256, 256]) torch.Size([16, 1, 256, 256])
torch.Size([2, 2, 256, 256]) torch.Size([2, 1, 256, 256])
torch.Size([16, 2, 256, 256]) torch.Size([16, 1, 256, 256])
torch.Size([16, 2, 256, 256]) torch.Size([16, 1, 256, 256])
torch.Size([16, 2, 256, 256]) torch.Size([16, 1, 256, 256])
torch.Size([2, 2, 256, 256]) torch.Size([2, 1, 256, 256])
torch.Size([16, 2, 256, 256]) torch.Size([16, 1, 256, 256])
torch.Size([16, 2, 256, 256]) torch.Size([16, 1, 256, 256])
torch.Size([16, 2, 256, 256]) torch.Size([16, 1, 256, 256])
torch.Size([2, 2, 256, 256]) torch.Size([2, 1, 256, 256])


KeyboardInterrupt: ignored

In [ ]:
for i in range(1):

    landscape = inverse_transform(X[i]).permute(1, 2, 0).cpu().detach().numpy()
    label_class = Y[i].cpu().detach().numpy()
    label_class_predicted = Y_pred[i].cpu().detach().numpy()

    axes[i, 0].imshow(landscape)
    axes[i, 0].set_title("Landscape")
    axes[i, 1].imshow(label_class)
    axes[i, 1].set_title("Label Class")
    axes[i, 2].imshow(label_class_predicted)
    axes[i, 2].set_title("Label Class - Predicted")

In [ ]:
# Update with your actual directories
test_img_dir = '/content/Segmentation01/test/org'
test_mask_dir = '/content/Segmentation01/test/label'

# Create the dataset
test_dataset = LungSegmentationDataset(test_img_dir, test_mask_dir, transform=transform)
# Define data loader
batch_size = 16  # Adjust based on your resources and dataset size
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)




In [ ]:
for epoch in range(num_epochs):
    total_loss = 0
    for i, (images, masks) in enumerate(test_loader):
        images = images.to(device)
        masks = masks.to(device)
        # images = images.unsqueeze(0)
        # masks = masks.unsqueeze(0)
        print(images.shape,masks.shape)
        # Forward pass
        outputs = model(images)
        print(outputs.shape,masks.shape)
        masks = masks.squeeze(1)
        masks = masks.long()
        loss = criterion(outputs, masks)
        total_loss += loss.item()
        print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
average_loss = total_loss / len(train_loader)
print(f'Epoch [{epoch+1}/{num_epochs}], Average Loss: {average_loss:.4f}')

torch.Size([10, 1, 256, 256]) torch.Size([10, 1, 256, 256])
torch.Size([10, 2, 256, 256]) torch.Size([10, 1, 256, 256])
Epoch [1/10], Step [1/4], Loss: 0.1462
torch.Size([10, 1, 256, 256]) torch.Size([10, 1, 256, 256])
torch.Size([10, 2, 256, 256]) torch.Size([10, 1, 256, 256])
Epoch [2/10], Step [1/4], Loss: 0.1462
torch.Size([10, 1, 256, 256]) torch.Size([10, 1, 256, 256])
torch.Size([10, 2, 256, 256]) torch.Size([10, 1, 256, 256])
Epoch [3/10], Step [1/4], Loss: 0.1462
torch.Size([10, 1, 256, 256]) torch.Size([10, 1, 256, 256])
torch.Size([10, 2, 256, 256]) torch.Size([10, 1, 256, 256])
Epoch [4/10], Step [1/4], Loss: 0.1462
torch.Size([10, 1, 256, 256]) torch.Size([10, 1, 256, 256])
torch.Size([10, 2, 256, 256]) torch.Size([10, 1, 256, 256])
Epoch [5/10], Step [1/4], Loss: 0.1462
torch.Size([10, 1, 256, 256]) torch.Size([10, 1, 256, 256])
torch.Size([10, 2, 256, 256]) torch.Size([10, 1, 256, 256])
Epoch [6/10], Step [1/4], Loss: 0.1462
torch.Size([10, 1, 256, 256]) torch.Size([10, 

In [ ]:
X, Y = next(iter(test_loader))
X, Y = X.to(device), Y.to(device)
Y_pred = model(X)
print(Y_pred.shape)
Y_pred = torch.argmax(Y_pred, dim=1)
print(Y_pred.shape)

torch.Size([10, 2, 256, 256])
torch.Size([10, 256, 256])


In [ ]:
import matplotlib.pyplot as plt

inverse_transform = transforms.Compose([
    transforms.Normalize(-0.485 / 0.229, 1 / 0.229)  # Single mean and std for a grayscale image
])


fig, axes = plt.subplots(1, 3, figsize=(3*5, 1*5))



for x,y in test_loader:
  x, y = x.to(device), y.to(device)
  y_pred = model(x)
  y_pred = torch.argmax(y_pred,dim = 1)
  landscape = inverse_transform(x[i]).permute(1, 2, 0).cpu().detach().numpy()
  label_class = y[i].cpu().detach().numpy()
  label_class_predicted = y_pred[i].cpu().detach().numpy()

  axes[0].imshow(landscape,cmap='gray')
  axes[0].set_title("Landscape")
  axes[1].imshow(label_class[0],cmap='gray')
  axes[1].set_title("Label Class")
  axes[2].imshow(label_class_predicted,cmap='gray')
  axes[2].set_title("Label Class - Predicted")


In [ ]:
import matplotlib.pyplot as plt

inverse_transform = transforms.Compose([
    transforms.Normalize(-0.485 / 0.229, 1 / 0.229)  # Single mean and std for a grayscale image
])


fig, axes = plt.subplots(1, 3, figsize=(3*5, 1*5))

for i in range(1):

    landscape = inverse_transform(X[i]).permute(1, 2, 0).cpu().detach().numpy()
    label_class = Y[i].cpu().detach().numpy()
    label_class_predicted = Y_pred[i].cpu().detach().numpy()

    axes[0].imshow(landscape,cmap='gray')
    axes[0].set_title("Landscape")
    axes[1].imshow(label_class[0],cmap='gray')
    axes[1].set_title("Label Class")
    axes[2].imshow(label_class_predicted)
    axes[2].set_title("Label Class - Predicted")